# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQJ986.QJ952.7.8'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
models.ns = 1
models.ew = 2
auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '2C',
  'insta_score': 0.6512,
  'expected_score': -90,
  'adjustment': 33},
 {'call': 'PASS',
  'insta_score': 0.2068,
  'expected_score': -119,
  'adjustment': 10}]

In [4]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.9852,
  'expected_score': -52,
  'adjustment': 49},
 {'call': '2S',
  'insta_score': 0.0126,
  'expected_score': -7,
  'adjustment': -20}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['xx.ATx.AKT.AKQJx Kx.Kx.QJ98xx.9xx Txx.8xx.xxx.Txxx AQJ98x.QJ9xx.x.8 0.94234 2D-P-P-X-P-P-P (8.0)  2D-P-P-X-P-P-2S-3C-P-P-P (8.3) ',
 'Kx.AKT8.x.AKQxxx xxx.x.AJ98xx.J9x Tx.xxx.KQTxx.Txx AQJ98x.QJ9xx.x.8 0.93737 2D-P-P-X-P-P-P (6.7)  2D-P-P-X-P-P-2S-3C-P-P-P (9.9) ',
 'Kxx.Ax.AJ.AKJxxx T.KT8.KT98xx.T9x xxx.xxx.Qxxx.Qxx AQJ98x.QJ9xx.x.8 0.93234 2D-P-P-X-P-P-P (7.2)  2D-P-P-X-P-P-2S-3C-P-P-P (9.0) ',
 'Kxx.Axx.Kx.AKQTx Tx.Kx.AQT98x.xxx xx.T8x.Jxxx.J9xx AQJ98x.QJ9xx.x.8 0.92488 2D-P-P-X-P-P-P (8.1)  2D-P-P-X-P-P-2S-3C-P-P-P (8.1) ',
 'xxx.AKT8.Tx.AKJx Tx.x.QJ98xx.Qxxx Kx.xxx.AKxx.T9xx AQJ98x.QJ9xx.x.8 0.82310 2D-P-P-X-P-P-P (6.5)  2D-P-P-X-P-P-2S-P-P-P (6.9) ',
 'Kxx.AKTx.Jx.A9xx x.xx.AQTxxx.xxxx Txx.8x.K98x.KQJT AQJ98x.QJ9xx.x.8 0.81570 2D-P-P-X-P-P-P (7.0)  2D-P-P-X-P-P-2S-P-2N-P-P-P (5.6) ',
 'Kxx.AKTx.A8.AKxx x.x.KQJxxx.QJT9x Txx.8xx.T9xx.xxx AQJ98x.QJ9xx.x.8 0.77768 2D-P-P-X-P-P-P (7.5)  2D-P-P-X-P-P-2S-X-P-P-P (7.5) ',
 'Kxx.AK8x.KJ.Jxxx T.Tx.ATxxxx.KQTx xxx.xx.Q98x.A9xx AQJ98x.QJ9x

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

AQJ98x.QJ9xx.x.8 Txx.xxx.AKT.ATxx .KT8x.QJ9xx.KQJ9 Kxxx.A.8xxx.xxxx
AQJ98x.QJ9xx.x.8 Kxx.Tx.J9xx.KJT9 T.A8xx.AK8x.xxxx xxx.Kx.QTxx.AQxx
AQJ98x.QJ9xx.x.8 Kxx.xx.KQTx.QJxx x.A8xx.A8xx.KTxx Txx.KT.J9xx.A9xx
AQJ98x.QJ9xx.x.8 Txx.Ax.QJ8x.AQ9x x.KTxx.AKT9.JTxx Kxx.8x.xxxx.Kxxx
AQJ98x.QJ9xx.x.8 Kxx.K8x.xxx.JTxx x.Axxx.K98x.AKxx Txx.T.AQJTx.Q9xx
AQJ98x.QJ9xx.x.8 Tx.Tx.AKJxxx.T9x x.AKxx.T9x.KQxxx Kxxx.8x.Q8x.AJxx
AQJ98x.QJ9xx.x.8 xxx.A8x.AQT8x.xx x.KTxx.K9x.A9xxx KTx.x.Jxxx.KQJTx
AQJ98x.QJ9xx.x.8 Kxx.x.AT8x.QJxxx x.KT8x.KJxx.AK9x Txx.Axx.Q9xx.Txx
AQJ98x.QJ9xx.x.8 Kx.Ax.JTx.Jxxxxx x.Kxxx.KQxx.AKTx Txxx.T8.A98xx.Q9
AQJ98x.QJ9xx.x.8 Kxx.8xx.KQTx.Jxx x.AKTx.J9xx.AQxx Txx.x.A8xx.KT9xx
AQJ98x.QJ9xx.x.8 xx.Tx.KQ98x.KQ9x x.A8xx.Txxx.AJTx KTxx.Kx.AJx.xxxx
AQJ98x.QJ9xx.x.8 KT.Txxx.KJxx.KTx x.AK8x.QT9x.AQ9x xxxx..A8xx.Jxxxx
AQJ98x.QJ9xx.x.8 Kxxx.xx.ATxx.Kxx x.AKTx.J9x.AQTxx Tx.8x.KQ8xx.J9xx
AQJ98x.QJ9xx.x.8 KTx.Tx.Qxxx.Axxx x.A8xx.AK8.KJ9xx xxx.Kx.JT9xx.QTx
AQJ98x.QJ9xx.x.8 xx.T8.AJTx.ATxxx x.AKxx.98xxx.K